## Образец заданий из зачета

_Тут не все решения, ибо автору стало лень_  
_но на зачет 60% хватит_

### задача 1

Полностью соотвествует первому упражнению

SELECT * FROM pulsar_stars WHERE ((TARGET = 0 AND MIP >=83 AND MIP <=84) OR (TARGET = 1 AND MIP >=83 AND MIP <=89)) 

**Первый ответы** это число строк в базе данных

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv("report.csv")
df

,MIP,STDIP,EKIP,SIP,MC,STDC,EKC,SC,TARGET
0,84.742188,40.007274,1.368957,3.812073,5.736622,32.047311,5.979603,36.147975,1
1,83.679688,36.379281,0.572532,2.664611,4.040970,23.169129,7.006681,53.514005,0
2,83.968750,32.786939,0.972955,4.940515,1.638796,11.934603,12.157317,199.321879,0
3,83.531250,42.521674,0.581761,1.230803,3.428094,17.796434,7.050442,61.662594,0
4,88.437500,37.811777,0.689216,2.035294,2.415552,18.027953,8.828613,86.799894,1
...,...,...,...,...,...,...,...,...,...
131,84.054688,51.992083,0.818566,0.037858,158.000836,95.239274,-0.799085,-1.044223,1
132,83.710938,37.365415,0.659945,2.423515,4.178094,23.202703,6.532946,46.378170,0
133,84.695312,49.348499,0.960378,1.102570,5.490803,27.764100,6.181768,40.399247,1
134,83.773438,52.279031,1.206698,1.630834,97.373746,90.905524,0.132639,-1.683188,1


**Второй ответ** тут уже придется запомнить, но  
Сверху мы импортировали MinMaxScaler  
А снизу просто создали новую бд

In [3]:
scaler = MinMaxScaler() # Дали функции имя
names = df.columns # Запомнили названия колонок
d = scaler.fit_transform(df) #Преобразовали данные
scaled_df = pd.DataFrame(d, columns=names) #Создали бд использовав даные до этого

scaled_df["MIP"].mean() # Высилили выборочное среднее

0.26595638255302184

Тут уже берется 3 упражнение

In [4]:
from sklearn.linear_model import LogisticRegression # Импортируем

In [5]:
X = pd.DataFrame(scaled_df.drop(['TARGET'], axis=1)) # Сюда заносим уже обработанную базу но без TARGET
Y = pd.DataFrame(scaled_df['TARGET']) # Сюда только TARGET

In [6]:
reg = LogisticRegression(random_state=2019, solver='lbfgs').fit(X, Y.values.ravel())

**Третий ответ**  
Первое значение "не пульсар"

In [7]:
reg.predict_proba([[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]]) # Здесь вводим данные звезды

array([[0.72644503, 0.27355497]])

In [8]:
from sklearn.neighbors import KNeighborsClassifier # Импортируем

Запускаем те же команды, только для Y сразу ставим values.ravel

In [9]:
X2 = pd.DataFrame(scaled_df.drop(['TARGET'], axis=1))
Y2 = pd.DataFrame(scaled_df['TARGET']).values.ravel()

Тут запоминаем, что надо ставить p=1 это манхетенское

In [10]:
Manhattan = KNeighborsClassifier(n_neighbors=3, p=1)
Manhattan.fit(X2, Y2)

KNeighborsClassifier(n_neighbors=3, p=1)

Отсюда берем первое значение

In [11]:
Manhattan.kneighbors([[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]])

(array([[0.5479    , 0.56068203, 0.57204809]]),
 array([[122, 132,   1]], dtype=int64))

### Задача 2

In [ ]:
!pip install --upgrade pip
!pip install --upgrade scikit-learn==0.23.0

In [12]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
from keras.datasets import mnist
(X_train, y_train), (X_pred, y_pred) = mnist.load_data()

In [14]:
dim = 784 # 28*28 
X_train_reshape = X_train.reshape(len(X_train), dim)

In [15]:
from sklearn.decomposition import PCA
pca = PCA(svd_solver='full')
pca = pca.fit(X_train_reshape) 

**Первый ответ второй задачи**  
_Тут уже хватит на зачет_

In [16]:
M = np.cumsum(pca.explained_variance_ratio_)
for i, value in enumerate(M):
    if (value>0.83):
        print(i+1)
        break

52


In [17]:
from sklearn.model_selection import train_test_split
X_train = X_train.reshape(len(X_train), dim)

In [18]:
pca = PCA(n_components=52, svd_solver='full')
pca = pca.fit(X_train)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=126)

In [19]:

X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

**Второй ответ**  
_Тут уже придется решать предыдущее, чтобы найти_

In [20]:
np.mean(X_train[:, 0])

0.5965431171660271

In [21]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [22]:
tree = RandomForestClassifier(criterion='gini', min_samples_leaf=10, max_depth=20, n_estimators=10, random_state=126)
clf = OneVsRestClassifier(tree).fit(X_train, y_train)

In [23]:
y_pred = clf.predict(X_test)

In [24]:
from sklearn.metrics import confusion_matrix
CM = confusion_matrix(y_test, y_pred)

In [25]:
CM

array([[1749,    0,    9,    5,    5,   12,   19,    5,   10,    3],
       [   3, 1993,   12,    8,    0,    7,    7,    4,    7,    7],
       [  23,    7, 1612,   24,   17,    6,   15,   24,   43,   13],
       [   6,    5,   35, 1588,    5,   54,   13,   22,   43,   14],
       [   4,    7,    8,    2, 1624,    2,   13,    7,   17,   61],
       [  18,    3,   12,   57,   16, 1414,   31,    4,   30,   25],
       [  15,    4,   16,    2,    6,   26, 1660,    0,    6,    1],
       [   5,   19,   25,    4,   21,    4,    1, 1770,    7,   45],
       [  15,   24,   30,   60,   14,   56,   10,   13, 1557,   25],
       [  12,    9,   11,   28,   63,   14,    1,   44,   26, 1562]],
      dtype=int64)

**Третий ответ**  
_Надо вставить значение, которое ищешь в 2 ячейки

In [26]:
CM[5][5]

1414